In [1]:
import mojito
import pprint
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from functools import lru_cache
from joblib import Memory


### Setup:
key = "API-KEY"
secret = "API-SECRET"
acc_no= "ACC-NO"

broker = mojito.KoreaInvestment(api_key=key, api_secret=secret, acc_no=acc_no)

memory = Memory("cache_directory", verbose=0)



In [2]:
#  Function to retrieve korean kospi and kosdaq stocks
def get_korean_stocks():
  # feth kospi symbols
  kospi = broker.fetch_kospi_symbols()
  kospi = pd.DataFrame(kospi)

  # extract 시가총액, 단축코드, 표준코드, and 한글명 columns
  kospi = kospi[['단축코드', '표준코드', '한글명', '시가총액']]

  # sort by 시가총액
  kospi = kospi.sort_values(by='시가총액', ascending=False)
  kospi['단축코드'] = kospi['단축코드'].apply(lambda x: x + '.KS')

  # do the same for kosdaq
  kosdaq = broker.fetch_kosdaq_symbols()
  kosdaq = pd.DataFrame(kosdaq)

  #add .ks to 단축코드 
  kosdaq['단축코드'] = kosdaq['단축코드'].apply(lambda x: x + '.KQ')

  # extract 시가총액, 단축코드, 표준코드, and 한글명 columns
  kosdaq = kosdaq[['단축코드', '표준코드', '한글명', '시가총액']]
  kosdaq = kosdaq.sort_values(by='시가총액', ascending=False)

  # add kospi and kosdaq into one dataframe
  korean_stock = pd.concat([kospi, kosdaq])
  korean_stock = korean_stock.reset_index(drop=True)
  korean_stock = korean_stock.sort_values(by='시가총액', ascending=False)

  return korean_stock


In [4]:
# Get beta values for each stock
@lru_cache(maxsize=None)
def get_beta_list(n = 500, stocks=None):
    try:
        if stocks is None:
            stocks = get_korean_stocks()

        # Ensuring the stocks is a list of tickers early on
        if not isinstance(stocks, list):
            stocks = stocks['단축코드'].tolist()
        else: 
            stocks = stocks

        # Limiting to the first n stocks if necessary
        stocks = stocks[:n]

        # Initialize an empty dictionary to store the volatilities
        volatilities = {}

        # Iterating over the stocks and fetching the beta value
        for ticker in stocks:
            try:
                beta = yf.Ticker(ticker).info.get('beta', 1)
            except Exception:
                beta = 1
            volatilities[ticker] = beta

        # Sorting the volatilities dictionary based on the absolute difference from 1 and filtering out those with a beta value of 1
        sorted_volatilities = sorted([(k, v) for k, v in volatilities.items() if v != 1], key=lambda item: abs(1 - item[1]), reverse=True)

        # Creating a dataframe from the sorted volatilities list of tuples
        volatilities_df = pd.DataFrame(sorted_volatilities, columns=['Ticker', 'Beta'])

        return volatilities_df

    except pd.errors.BaseException as pd_err:
        print(f"Pandas Error: {pd_err}")
    except Exception as e:
        print(f"An unknown error occurred: {e}")
        return pd.DataFrame()  # Returning an empty dataframe in case of an unknown error






In [5]:
def moving_average_crossover(data, short_window=50, long_window=200):
    # calculate moving averages
    short_mavg = data.rolling(window=short_window, min_periods=1, center=False).mean()
    long_mavg = data.rolling(window=long_window, min_periods=1, center=False).mean()

    # create signals
    signals = pd.DataFrame(index=data.index)

    # create signals
    signals['short_mavg'] = short_mavg
    signals['long_mavg'] = long_mavg
    signals['signal'] = 0.0
    try:
      signals['signal'][short_window:] = np.where(short_mavg[short_window:] > long_mavg[short_window:], 1.0, 0.0)
    except ValueError as e:
      print(f"An error occurred: {e}")

    # generate trading orders
    signals['positions'] = signals['signal'].diff()

    return signals


def calculate_returns(data, signals):
    if 'Adj Close' not in data.columns:
        raise ValueError("The input data DataFrame must contain an 'Adj Close' column.")
    
    # use the moving average crossover strategy to generate orders
    data['Order'] = signals['signal']

    # calculate daily and strategy returns
    data['Returns'] = data['Adj Close'].pct_change()
    data['Strategy Returns'] = data['Returns'] * data['Order'].shift(1)
    data['Cumulative Returns'] = (1 + data['Strategy Returns']).cumprod()

    return data


def plot_cumulative_returns(data):
    # plot cumulative returns
    plt.figure(figsize=(5,5))
    plt.title('Cumulative Returns')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    data['Cumulative Returns'].plot(lw=2.)
    ((data['Returns']+1).cumprod()).plot(lw=2.)
    plt.show()



def create_portfolio(signals, data, initial_capital=100000.0):
    # create a dataframe `positions`
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    positions['stock'] = 100 * signals['signal']

    adj_close = data['Adj Close']
    positions_value = positions.multiply(adj_close, axis=0)
    pos_diff = positions.diff()

    # initialize the portfolio with value owned
    portfolio = positions_value

    # store the difference in shares owned
    pos_diff = positions.diff()

    # add `holdings` to portfolio
    portfolio['holdings'] = positions_value.sum(axis=1)

    # add `cash` to portfolio
    portfolio['cash'] = initial_capital - pos_diff.multiply(adj_close, axis=0).sum(axis=1).cumsum()

    # add `total` to portfolio
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']

    # add `returns` to portfolio
    portfolio['returns'] = portfolio['total'].pct_change()

    return portfolio


def plot_portfolio_performance(portfolio, signals):
    # plot the portfolio value over time
    plt.figure(figsize=(5,5))
    ax1 = plt.subplot(111, ylabel='Portfolio value in $')
    plt.title('Portfolio Performance')
    plt.xlabel('Date')

    # plot the equity curve in dollars
    portfolio['total'].plot(ax=ax1, lw=2.)

    # plot the "buy" trades against the equity curve
    ax1.plot(portfolio.loc[signals.positions == 1.0].index,
             portfolio.total[signals.positions == 1.0],
             '^', markersize=10, color='g')

    # plot the "sell" trades against the equity curve
    ax1.plot(portfolio.loc[signals.positions == -1.0].index,
             portfolio.total[signals.positions == -1.0],
             'v', markersize=10, color='r')

    plt.show()

@memory.cache
def trade(weights = None, 
          initial_capital = 100000.0, 
          years = 5, 
          plot = False, 
          stocks = None, 
          start_date = None, 
          end_date = None):
    
    # initialize stock trade dictionary
    stock_trades = {}

    # dictionary to store daily returns of each stock
    daily_returns = {}

    if stocks is None:
       stocks = get_beta_list()
       stocks = stocks['Ticker']

    if end_date is None: # end_date is today
        end_date = datetime.now().strftime('%Y-%m-%d')

    if start_date is None: # start_date is 5 years ago
      start_date = (datetime.now() - timedelta(days=years*365)).strftime('%Y-%m-%d')

    for stock in stocks:
      # download data from yfinance
      data = yf.download(stock, start=start_date, end=end_date, progress=False)

      if data.empty:
            print(f"No data found for stock: {stock}")
            continue

      try:
          # run moving average crossover strategy
          signals = moving_average_crossover(data['Adj Close'])
      except KeyError as e:
          print(f"KeyError occurred while processing stock '{stock}': {e}. Skipping...")
          continue


      # calculate returns
      data = calculate_returns(data, signals)

      # Storing daily returns in the dictionary
      try:
          # Storing daily returns in the dictionary
          daily_returns[stock] = data['Returns'].dropna()
      except KeyError as e:
          print(f"KeyError occurred while processing stock '{stock}': {e}. Skipping...")
          continue


      if weights is not None:
        try:
            allocated_capital = weights.loc[stock]['Weight'] * initial_capital
        except KeyError as e:
            print(f"KeyError occurred while processing stock '{stock}': {e}. Using default allocated capital...")
            allocated_capital = initial_capital
      else:
        allocated_capital = initial_capital

      # create portfolio
      portfolio = create_portfolio(signals, data, allocated_capital)

      # if plot is True, plot cumulative returns and portfolio performance
      if plot:
          # plot cumulative returns
          plot_cumulative_returns(data)
          # plot portfolio performance
          plot_portfolio_performance(portfolio, signals)

      # print the final cumulative return
      try:
          # print the final cumulative return
          cumulative_return = data['Cumulative Returns'][-1]
      except KeyError as e:
          print(f"KeyError occurred while processing stock '{stock}': {e}. Skipping...")
          continue

      # print return
      final_total_return = ((portfolio['total'].iloc[-1] / portfolio['total'].iloc[0]) - 1) * 100

      # update dictionary
      stock_trades[stock] = {'Final Total Return': final_total_return, 'Cumulative Return': cumulative_return}
    
    stock_trades = pd.DataFrame(stock_trades).T
    stock_trades.sort_values(by=['Final Total Return'], ascending=False, inplace=True)

    daily_returns = pd.DataFrame(daily_returns)

    return stock_trades, daily_returns

In [6]:
def max_sharpe_ratio_portfolio(num_portfolios = 1000000, returns = None):
    if returns is None:
      stock_trades, daily_returns = trade()

      # profitable stocks
      profitable_stocks = stock_trades[stock_trades['Cumulative Return'] > 2].index

      # extract daily returns of profitable stocks
      profitable_daily_returns = daily_returns[profitable_stocks]

      # replace NaN values with 0
      profitable_daily_returns.fillna(0, inplace=True)

      returns = profitable_daily_returns

    # Get the number of assets
    num_assets = len(returns.columns)
    
    # Get the mean and covariance of the returns
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    
    # Initialize lists to store the simulation results
    results = np.zeros((3, num_portfolios))
    weight_array = []
    
    for i in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        weight_array.append(weights)
        
        # Expected portfolio return
        portfolio_return = np.dot(weights, mean_returns)
        
        # Expected portfolio volatility
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        
        # Portfolio Sharpe ratio
        portfolio_sharpe = portfolio_return / portfolio_stddev
        
        results[0,i] = portfolio_return
        results[1,i] = portfolio_stddev
        results[2,i] = portfolio_sharpe
    
    # Convert results array to DataFrame
    results_frame = pd.DataFrame(results.T, columns=['Return','Volatility','Sharpe'])
    
    # Extract the portfolio with the max Sharpe ratio
    max_sharpe_idx = results_frame['Sharpe'].idxmax()
    max_sharpe_portfolio = pd.Series(weight_array[max_sharpe_idx], index=returns.columns)

    # change it to dataframe
    max_sharpe_portfolio = pd.DataFrame(max_sharpe_portfolio, columns=['Weight'])

    return max_sharpe_portfolio


In [7]:
# Trade according to the max sharpe ratio portfolio
def portfolio_trade(plot: bool = False, initial_capital: int = 10000, stocks = None, returns = None, start_date = None, end_date = None):
    if stocks and returns is None:
        stock_trades, daily_returns = trade()

        # stocks are profitable stocks
        stocks = stock_trades[stock_trades['Cumulative Return'] > 2].index

        # extract daily returns of profitable stocks
        profitable_daily_returns = daily_returns[stocks]

        # replace NaN values with 0
        profitable_daily_returns.fillna(0, inplace=True)
        returns = profitable_daily_returns 

    if start_date is None: # start_date is 5 years ago
      start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')
    
    if end_date is None:
      end_date = datetime.now().strftime('%Y-%m-%d')
    
    # Alloate each stock with weights according to max sharpe ratio portfolio
    weights = max_sharpe_ratio_portfolio(1000000, returns = returns)
    
    # stocks are only stocks with weights in the max sharpe ratio portfolio
    stocks = get_beta_list()['Ticker'].drop([i for i, x in enumerate(get_beta_list()['Ticker']) if x not in weights.index])
        
    # trade the stocks
    trades, _ = trade(weights = weights, 
          initial_capital = 100000, 
          years = 5, 
          plot = plot, 
          stocks = stocks, 
          start_date = start_date, 
          end_date = end_date)
    
    # find total value of the portfolio
    def total_value(trades):
      total_value = trades['Final Total Return'].sum()
      return total_value

    # calculate percentage change in total value
    def percentage_change(trades):
      total_value = trades['Final Total Return'].sum()
      percentage_change = (total_value - initial_capital) / initial_capital * 100
      # format it as percentage
      percentage_change = "{:.2f}%".format(percentage_change)
      return percentage_change  
    
    # create dataframe to store total value and percentage change
    stats = {'Total Value': total_value(trades), 'Percentage Change': percentage_change(trades)}
    stats = pd.DataFrame(stats, index=[0])

    return weights, trades, stats




